<h1 align="center"> IXPDB HTML scraper </h1>

Author:

    - Joao M. Ceron (SIDNLabs)

In [576]:
import pandas as pd
import cfscrape
from lxml import etree
from bs4 import BeautifulSoup
import re

### INSTANTIATING THE CRAWLER AND DEFINING AN REALISTIC HEADER


In [577]:
scraper = cfscrape.create_scraper()
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'}

In [578]:
url ="https://api.ixpdb.net/v1/provider/list"
scraped_html = scraper.get(url,headers=header).content
html = etree.HTML(scraped_html)

### PREPARE DATAFRAME

In [579]:
df_all_ixp = pd.read_json(scraped_html)
df_all_ixp = df_all_ixp[['id','name','city','country','participant_count','apis']].fillna(0)
def get_url(x):
    api = x.get('ixfexport')
    if (api):
        return (str(api))
    return (None)

df_all_ixp['api'] = df_all_ixp['apis'].apply(get_url)
# filter IXPs with info from 1 or more participants
df_all_ixp = df_all_ixp[(df_all_ixp.participant_count>0) & (~df_all_ixp.api.isnull())]
df_all_ixp.drop('apis',axis=1,inplace=True)
df_all_ixp

,id,name,city,country,participant_count,api
4,323,ACT-IX (IX Australia (Canberra ACT)),Canberra,AU,9,https://ix.asn.au/participants.json
6,586,"AKL-IX (New Zealand Internet Exchange, Auckland)",Auckland,NZ,87,https://www.ix.nz/participants.json
8,737,ams (Asteroid Amsterdam IX),Amsterdam,NL,33,https://sputnik.asteroidhq.com/export/euroix/1...
9,2,AMS-IX,Amsterdam,NL,868,https://stats.ams-ix.net/exports/members.json
10,61,AMS-IX Caribbean,"Willemstad, Curacao",AN,12,https://stats.ams-ix.net/exports/members.json
...,...,...,...,...,...,...
608,393,VIC-IX (IX Australia (Melbourne VIC)),Melbourne,AU,108,https://ix.asn.au/participants.json
609,3,VIX (Vienna Internet Exchange),,AT,149,https://www.vix.at/participants.json
610,170,vlv-ix (VLV-IX public peering VLAN),Vladivostok,RU,19,https://www.msk-ix.ru/data/json/ixp-member-lis...
613,38,VSIX (VSIX Nap del Nord Est),,IT,49,https://www.vsix.it/participants.json


### EXTRACT INFO FROM IXP PARTICIPANTS

In [147]:
df_all = pd.DataFrame([])
array_ixp = []
total = len(df_all_ixp.id)
counter = 0
for id in df_all_ixp.id:
    counter = counter + 1
    url = "https://ixpdb.euro-ix.net/en/ixpdb/ixp/{}/asns/".format(id)
    print (url) 
    print ("{}/{}".format(counter,total))
    
    scraped_html = scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)
    soup = BeautifulSoup(scraped_html, 'lxml')
    tab = soup.find("table",{"class":"table table-striped"})
    try:
        rows = tab.find_all('tr')
    except:
        next
    all_rows = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        rows_list = []
        count = 0
        for ele in cols:
            count = count +1 
            ele=ele.strip()
            pattern = re.compile(r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})') 
            if (pattern.search(ele)):
                ele = pattern.search(ele)[0]
            rows_list.append(ele)            
        if (rows_list):
            all_rows.append(rows_list)

    df1 = pd.DataFrame(all_rows)
    df1.columns = ['peer', 'peer_name', 'type', 'ip', 'mac','mars','peeringdb','asns_peering_with']
    df1['ixp'] = str(id)
    df_all = pd.concat([df_all,df1])

https://ixpdb.euro-ix.net/en/ixpdb/ixp/323/asns/
1/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/586/asns/
2/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/737/asns/
3/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/2/asns/
4/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/61/asns/
5/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/356/asns/
6/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/619/asns/
7/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/532/asns/
8/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/533/asns/
9/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/746/asns/
10/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/564/asns/
11/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/741/asns/
12/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/587/asns/
13/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/21/asns/
14/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/855/asns/
15/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/12/asns/
16/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/55/asns/
17/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/40/asns/
18/240
https://

https://ixpdb.euro-ix.net/en/ixpdb/ixp/707/asns/
147/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/515/asns/
148/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/796/asns/
149/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/626/asns/
150/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/627/asns/
151/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/706/asns/
152/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/710/asns/
153/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/708/asns/
154/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/95/asns/
155/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/764/asns/
156/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/96/asns/
157/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/49/asns/
158/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/800/asns/
159/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/78/asns/
160/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/275/asns/
161/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/845/asns/
162/240
https://ixpdb.euro-ix.net/en/ixpdb/ixp/1/asns/
163/240
https://ixpdb.euro-ix.net/en/ixpdb/ix

### GET A LIST OF MACs PER IXP (WHEN AVAILABLE)

In [580]:
df_macs = df_all[df_all['mac']!=""].groupby('ixp').mac.apply(list).reset_index()
df_macs.columns=["id","mac"]
df_macs['id'] = df_macs.id.astype(int)
df_macs.head(1)

,id,mac
0,1,"[6c:9c:ed:30:11:ab, 88:f0:31:ad:f7:70, 2c:21:7..."


### FILTER AND PREPARE DATAFRAME FORMAT

In [582]:
df = pd.merge(df_all_ixp,df_macs,on="id")
for id in df.id.unique():
    mac_list =  (df[df.id==id]['mac'].tolist())
    flat_list = [item for sublist in mac_list for item in sublist]
    str_list = ",".join(flat_list)
    mac_list = str_list.split()
    df.loc[df.id==id,'mac_list'] = str(mac_list)
    df.loc[df.id==id,'len'] = int(len(mac_list))
df.drop('mac',axis=1,inplace=True)
df.head()

,id,name,city,country,participant_count,api,mac_list,len
0,746,ANIX (Albanian Neutral Internet eXchange),,AL,11,https://anix.al/export.json,['0c:45:ba:9d:8c:1b'],1.0
1,741,BALT-IX (Baltic Internet Exchange),,LT,50,http://stats.balt-ix.net/members/0.7/participa...,"['64:87:88:5c:43:b9,30:e4:db:34:2c:c0,b0:aa:77...",6.0
2,587,Bangkok Neutral Internet Exchange (BKNIX),,TH,36,https://bknix.co.th/data/public/members.json,"['00:0c:29:ca:94:8e', '24:e9:b3:66:25:50', '00...",24.0
3,21,BCIX (BCIX Management GmbH),,DE,110,https://www.bcix.de/ixp/api/v4/member-export/i...,"['88:5a:92:46:cf:d1,00:30:48:56:c8:05,00:25:90...",31.0
4,855,BFD-IX (BFD-IX),,GB,41,http://manager.bfd-ix.net/api/v4/member-export...,"['06:85:a3:d6:43:e3,82:73:a8:a5:05:79']",1.0


### FILTER IXPs with more than 20 MAC addresses

In [513]:
df = df[df['len']>20]
df.len.value_counts()

36.0     4
98.0     1
129.0    1
47.0     1
56.0     1
27.0     1
170.0    1
63.0     1
28.0     1
22.0     1
38.0     1
53.0     1
31.0     1
24.0     1
Name: len, dtype: int64

### INFER ACRONYMS FROM NAME

In [588]:
for id in df.id.unique():
    name = df[df.id==id]['name'].values
    acronyms = re.findall('\\b[A-Z](?:[\\.&]?[A-Z]){1,7}\\b', str(name))
    if not (acronyms):
        acronyms = "[]"
    df.loc[df.id==id,'acronyms'] = str(acronyms[0])
df.head(1)

,id,name,city,country,participant_count,api,mac_list,len,acronyms
0,746,ANIX (Albanian Neutral Internet eXchange),,AL,11,https://anix.al/export.json,['0c:45:ba:9d:8c:1b'],1.0,ANIX


### FIX ACRONYMS 

In [589]:
df.loc[df.name.str.contains(r"France-IX"),'acronyms']="France-IX"
df.loc[df.name.str.contains(r"SwissIX"),'acronyms']="SwissIX"
df.loc[df.name.str.contains(r"GR-IX"),'acronyms']="GR-IX"
df.loc[df.name.str.contains(r"Denver"),'acronyms']="Denver-IX"
df.loc[df.name.str.contains(r"IX-SYD "),'acronyms']="IX-SYD"
df.loc[df.name.str.contains(r"KleyReX"),'acronyms']="KleyReX"
df.loc[df.name.str.contains(r"NAPAfrica"),'acronyms']="NAPAfrica"
df[['name','acronyms']]

,name,acronyms
0,ANIX (Albanian Neutral Internet eXchange),ANIX
1,BALT-IX (Baltic Internet Exchange),BALT
2,Bangkok Neutral Internet Exchange (BKNIX),BKNIX
3,BCIX (BCIX Management GmbH),BCIX
4,BFD-IX (BFD-IX),BFD
...,...,...
71,tdc-ops-sw1 (SUPRnet),[
72,TR-IX (Turk IX),TR
73,TTPL LAN (Bhutan Internet Exchange),TTPL
74,UEPG Internet Exchange (UEPG IX),UEPG


### GROUP MACS BY IXP AND WRITE THEM ON A FILE  

In [574]:
for id in df.id.unique():
    filename = "./dataset/new/arp.{}.txt".format(df.loc[df.id==id,'acronyms'].values[0])
    print (filename)
    mac_list = df.loc[df.id==id,'mac_list'].values[0]
    mac_list = mac_list.split(",")
    with open(filename, "a") as myfile:
        for mac in mac_list:
            mac_search = re.search('.*((?:[0-9a-fA-F]:?){12}).*', mac, re.IGNORECASE)
            if mac_search:
                mac_ = mac_search.group(1)
                myfile.write(mac_)
                myfile.write("\n")
    # remove dup lines
    with open(filename) as fl:
        content = fl.read().split('\n')
    content = set([line for line in content if line != ''])
    content = '\n'.join(content)
    with open(filename, 'w') as fl:
        fl.writelines(content)

./dataset/new/arp.BKNIX.txt
./dataset/new/arp.BCIX.txt
./dataset/new/arp.EPIX.txt
./dataset/new/arp.EPIX.txt
./dataset/new/arp.France-IX.txt
./dataset/new/arp.GR-IX.txt
./dataset/new/arp.SwissIX.txt
./dataset/new/arp.Denver-IX.txt
./dataset/new/arp.IX-SYD.txt
./dataset/new/arp.KleyReX.txt
./dataset/new/arp.LINX.txt
./dataset/new/arp.LINX.txt
./dataset/new/arp.LINX.txt
./dataset/new/arp.LONAP.txt
./dataset/new/arp.NAPAfrica.txt
./dataset/new/arp.NAPAfrica.txt
./dataset/new/arp.STHIX.txt
